<a href="https://colab.research.google.com/github/Antonsen2/wildfire-ai/blob/automl_autogluon/autogluon_rapids_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Check #

First, click the _Runtime_ dropdown above in colab, at the top of the page, then _Change Runtime Type_ and confirm the instance type is _GPU_.

Next, check the output of `!nvidia-smi` to make sure you've been allocated a Tesla T4, P4, or P100.

In [1]:
!nvidia-smi

Tue Dec 13 08:57:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    26W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Environment Setup:
Set up script installs
1. Updates gcc in Colab
1. Installs Conda
1. Install RAPIDS' current stable version of its libraries, as well as some external libraries including:
1. Copy RAPIDS .so files into current working directory, a neccessary workaround for RAPIDS+Colab integration.


In [2]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/env-check.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 308, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 308 (delta 79), reused 98 (delta 55), pack-reused 171
Receiving objects: 100% (308/308), 89.88 KiB | 17.98 MiB/s, done.
Resolving deltas: 100% (141/141), done.
Traceback (most recent call last):
  File "rapidsai-csp-utils/colab/env-check.py", line 1, in <module>
    import pynvml
ModuleNotFoundError: No module named 'pynvml'


In [ ]:
# This will update the Colab environment and restart the kernel.  Don't run the next cell until you see the session crash.
!bash rapidsai-csp-utils/colab/update_gcc.sh
import os
os._exit(00)

Updating your Colab environment.  This will restart your kernel.  Don't Panic!
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [1,082 kB]
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 P

In [1]:
!python --version

Python 3.8.16


In [7]:
# This will install CondaColab.  This will restart your kernel one last time.  
# Run this cell by itself and only run the next cell once you see the session crash.
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [1]:
# Start running the rest of the cells as normal
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [4]:
#Installing RAPIDS
!pip install cudf-cu11 dask-cudf-cu11 --extra-index-url=https://pypi.ngc.nvidia.com
!pip install cuml-cu11 --extra-index-url=https://pypi.ngc.nvidia.com
!pip install cugraph-cu11 --extra-index-url=https://pypi.ngc.nvidia.com
!pip uninstall cupy-cuda115; pip install cupy-cuda11x

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 87.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 83.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 90.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.2/16.2 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.6/453.6 kB 32.2 MB/s 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 1.4/1.4 GB 34.8 MB/s eta 0:00:01tcmalloc: large alloc 1359798272 bytes == 0x555baa4e8000 @  0x7f9218ca21e7 0x555ba8248f72 0x555ba8257d5e 0x555ba829ba57 0x555ba830430b 0x555ba83041c3 0x555ba827db83 0x555ba825fc85 0x555ba825e351 0x555ba827026c 0x555ba825fc85 0x555ba825e351 0x555ba827026c 0x555ba825fc85 0x555ba825e351 0x555ba827026c 0x555ba825fc85 0x555ba825e351 0x555ba827026c 0x555ba825fc85 0x555ba825e351 0x555ba827026c 0x555ba825fc85 0x555ba82701d6 0x555ba825fc85 0x555ba825e351 0x555ba827026c 0x555ba82606fa 0x555ba825e351 0x555ba827026c 0x555ba825fc85
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 1.4/1.4 GB 34.7 MB/s eta 0:00:01tcmalloc: large alloc 1699749888 bytes == 0x555bfb5b6000 @  0x7f9218ca3615 0x555ba829ba57 0x555ba830430b 0x555ba83041c3 0x555ba827db83 0x555ba825fc85 0x555ba825e351 0x555ba827026c 0x555ba825f

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 1.0/1.0 GB 29.5 MB/s eta 0:00:01tcmalloc: large alloc 1285545984 bytes == 0x558094964000 @  0x7f13f25cb615 0x558054412a57 0x55805447b30b 0x55805447b1c3 0x5580543f4b83 0x5580543d6c85 0x5580543d5351 0x5580543e726c 0x5580543d6c85 0x5580543d5351 0x5580543e726c 0x5580543d6c85 0x5580543d5351 0x5580543e726c 0x5580543d6c85 0x5580543d5351 0x5580543e726c 0x5580543d6c85 0x5580543d5351 0x5580543e726c 0x5580543d6c85 0x5580543e71d6 0x5580543d6c85 0x5580543d5351 0x5580543e726c 0x5580543d76fa 0x5580543d5351 0x5580543e726c 0x5580543d6c85 0x5580543d5351 0x5580543e726c
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 GB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 1.6/1.6 GB 33.5 MB/s eta 0:00:01tcmalloc: large alloc 1627185152 bytes == 0x5580574c6000 @  0x7f13f25ca1e7 0x5580543bff72 0x5580543ced5e 0x558054

In [6]:
#Installing Autogluon
!pip install autogluon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
